In [1]:
## Metaclasses

In [2]:
class A:
    pass

In [3]:
type(A)

type

In [4]:
class Meta(type):
    pass

class MyClass(metaclass=Meta):
    pass

class MySubclass(MyClass):
    pass

In [5]:
type(MySubclass)

__main__.Meta

In [7]:
C = type("C", (), {})

In [8]:
type(C)

type

In [14]:
class DocsRequired(type):
    def __new__(cls, name, bases, dct):
        inst = super().__new__(cls, name, bases, dct)
        if not inst.__doc__:
            raise RuntimeError("documentation required")
        return inst

In [16]:
class A(metaclass=DocsRequired):
    """
    Some docs.
    """
    pass

In [17]:
## Descriptors

In [18]:
class A:
    v = 123

In [19]:
A.__dict__

mappingproxy({'__module__': '__main__',
              'v': 123,
              '__dict__': <attribute '__dict__' of 'A' objects>,
              '__weakref__': <attribute '__weakref__' of 'A' objects>,
              '__doc__': None})

In [20]:
a = A()

In [22]:
a.__dict__

{}

In [24]:
a.__class__.__dict__

mappingproxy({'__module__': '__main__',
              'v': 123,
              '__dict__': <attribute '__dict__' of 'A' objects>,
              '__weakref__': <attribute '__weakref__' of 'A' objects>,
              '__doc__': None})

Descriptors:

* `__get__`, `__set__`, `__delete__`

In [25]:
class A:
    x = 10

In [26]:
a = A()

In [27]:
a.x

10

In [28]:
class D:
    def __get__(self, obj, objtype=None):
        print("D.__get__", obj)
        return 10

In [29]:
class A:
    x = D()

In [30]:
a = A()

In [31]:
a.x

D.__get__ <__main__.A object at 0x73a3bf6c4740>


10

In [32]:
import os

In [41]:
class DirectorySize:
    def __get__(self, obj, objtype=None):
        return len(os.listdir(obj.dirname))

In [42]:
class Directory:

    size = DirectorySize()

    def __init__(self, dirname):
        self.dirname = dirname

In [43]:
d = Directory(".")

In [44]:
d.size

3

In [70]:
class IntRange:

    def __init__(self, min=None, max=None):
        self.min = min
        self.max = max
    
    def __set_name__(self, owner, name):
        self.private_name = "_" + name
    
    def __get__(self, obj, objtype):
        return getattr(obj, self.private_name)
    
    def __set__(self, obj, value):
        if self.min is not None:
            if value < self.min:
                raise ValueError("value too small")
        if self.max is not None:
            if value > self.max:
                raise ValueError("value too large")
        setattr(obj, self.private_name, value)
    


In [71]:
class Record:
    x = IntRange(min=10, max=20)
    y = IntRange(min=0)

In [72]:
record = Record()

In [73]:
record.__dict__

{}

In [74]:
record.x = 10

In [75]:
from abc import ABC, abstractmethod

class Validator(ABC):

    def __set_name__(self, owner, name):
        self.private_name = '_' + name

    def __get__(self, obj, objtype=None):
        return getattr(obj, self.private_name)

    def __set__(self, obj, value):
        self.validate(value)
        setattr(obj, self.private_name, value)

    @abstractmethod
    def validate(self, value):
        pass

class OneOf(Validator):

    def __init__(self, *options):
        self.options = set(options)

    def validate(self, value):
        if value not in self.options:
            raise ValueError(f'Expected {value!r} to be one of {self.options!r}')

class Number(Validator):

    def __init__(self, minvalue=None, maxvalue=None):
        self.minvalue = minvalue
        self.maxvalue = maxvalue

    def validate(self, value):
        if not isinstance(value, (int, float)):
            raise TypeError(f'Expected {value!r} to be an int or float')
        if self.minvalue is not None and value < self.minvalue:
            raise ValueError(
                f'Expected {value!r} to be at least {self.minvalue!r}'
            )
        if self.maxvalue is not None and value > self.maxvalue:
            raise ValueError(
                f'Expected {value!r} to be no more than {self.maxvalue!r}'
            )
        
class String(Validator):

    def __init__(self, minsize=None, maxsize=None, predicate=None):
        self.minsize = minsize
        self.maxsize = maxsize
        self.predicate = predicate

    def validate(self, value):
        if not isinstance(value, str):
            raise TypeError(f'Expected {value!r} to be an str')
        if self.minsize is not None and len(value) < self.minsize:
            raise ValueError(
                f'Expected {value!r} to be no smaller than {self.minsize!r}'
            )
        if self.maxsize is not None and len(value) > self.maxsize:
            raise ValueError(
                f'Expected {value!r} to be no bigger than {self.maxsize!r}'
            )
        if self.predicate is not None and not self.predicate(value):
            raise ValueError(
                f'Expected {self.predicate} to be true for {value!r}'
            )

In [76]:
class Component:

    name = String(minsize=3, maxsize=10, predicate=str.isupper)
    kind = OneOf('wood', 'metal', 'plastic')
    quantity = Number(minvalue=0)

    def __init__(self, name, kind, quantity):
        self.name = name
        self.kind = kind
        self.quantity = quantity

In [80]:
c = Component("DESK", "wood", 1)

In [81]:
c

## Iterator, Generator

In [82]:
for x in [1, 2, 3]:
    print(x)

1
2
3


In [85]:
for i in range(10):
    print(i)

0
1
2
3
4
5
6
7
8
9


In [86]:
[i * i for i in range(10)]

[0, 1, 4, 9, 16, 25, 36, 49, 64, 81]

In [97]:
g = (i * i for i in range(10))

In [99]:
for i in g:
    print(i)

0
1
4
9
16
25
36
49
64
81


In [117]:
g = os.walk("/home/tir")

In [118]:
len(g)

TypeError: object of type 'generator' has no len()

In [131]:
class Gen:

    def __iter__(self):
        return self

    def __next__(self):
        return 42
    

In [133]:
g = Gen()

In [134]:
next(g)

42

In [135]:
import itertools

In [136]:
for i in itertools.islice(g, 3):
    print(i)

42
42
42


In [143]:
class Gen:

    def __init__(self):
        self.i = 0
    
    def __iter__(self):
        return self
    
    def __next__(self):
        self.i += 1
        if self.i < 4:
            return self.i
        raise StopIteration

In [145]:
g = Gen()
for i in g:
    print(i)

1
2
3


In [139]:
next(g)

1

In [140]:
next(g)

2

In [141]:
next(g)

3

In [146]:
# yield

In [147]:
class Gen:

    def __init__(self):
        self.i = 0
    
    def __iter__(self):
        return self
    
    def __next__(self):
        self.i += 1
        if self.i < 4:
            return self.i
        raise StopIteration

In [155]:
def gen():
    while True:
        yield 42

In [156]:
gen()

<generator object gen at 0x73a3be6562a0>

In [157]:
g = gen()

In [160]:
for i in itertools.islice(g, 1):
    print(i)

42


In [176]:
class Gen: # class based approach

    def __init__(self):
        self.i = 0
    
    def __iter__(self):
        return self
    
    def __next__(self):
        self.i += 1
        if self.i < 4:
            return self.i
        raise StopIteration

def gen(): # with yield
    i = 0
    while i < 4:
        i += 1
        yield i


In [162]:
g = gen()

In [163]:
for i in g:
    print(i)

1
2
3
4


In [172]:
numbers = (i for i in range(1, 10))
squared = (i * i for i in numbers)
filtered = (i for i in squared if i % 7 == 0)

In [173]:
for i in filtered:
    print(i)

49


In [ ]:
def gen():
    i = 0
    while i < 4:
        i += 1
        yield i

Task:  Write a generator that mimicks the
builtin `range(start, stop, step)`.

* class-based generator
* yield based generator

```python
for i in Range(0, 10, 2):
    print(i)
    # 0, 2, 4, 6, 8
```

In [177]:
def range_func(start, stop, step):
    number = start
    while number < stop:
        yield number
        number += step

for i in range_func(0, 10, 2):
    print(i)

0
2
4
6
8


In [178]:
def myrange(stop, start=0, step=1):
    current = start
    while current < stop:
        yield current
        current += step

for i in myrange(5):
    print(i)

0
1
2
3
4


In [181]:
def myrangex(a, b=None, step=1):
    if b is None:
        start, stop = 0, a
    else:
        start, stop = a, b
    i = start
    while i < stop:
        yield i
        i += step

for i in myrangex(2, 4):
    print(i)

2
3


In [183]:
import sys
class genrange:
    def __init__(self,
                 start=0,
                 stop=sys.maxsize,
                 step=1):
        self.i = start
        self.stop = stop
        self.step = step
    def __iter__(self):
        return self
    def __next__(self):
        temp = self.i
        self.i += self.step
        if self.i <= self.stop:
            return temp
        else:
            raise StopIteration

for i in genrange(10,100, 11):
    print(i)

10
21
32
43
54
65
76
87


In [185]:
sys.maxsize

9223372036854775807

In [186]:
import itertools

In [194]:
for vs in itertools.product([True, False], repeat=4):
    print([int(v) for v in vs])

[1, 1, 1, 1]
[1, 1, 1, 0]
[1, 1, 0, 1]
[1, 1, 0, 0]
[1, 0, 1, 1]
[1, 0, 1, 0]
[1, 0, 0, 1]
[1, 0, 0, 0]
[0, 1, 1, 1]
[0, 1, 1, 0]
[0, 1, 0, 1]
[0, 1, 0, 0]
[0, 0, 1, 1]
[0, 0, 1, 0]
[0, 0, 0, 1]
[0, 0, 0, 0]


In [199]:
def fib(): # 1, 1, 2, 3, 5, 8, 13, ...
    a, b = 1, 0
    while True:
        yield a + b
        a, b = b, a + b


In [201]:
for i in itertools.islice(fib(), 0, 100, 10):
    print(i)


1
89
10946
1346269
165580141
20365011074
2504730781961
308061521170129
37889062373143906
4660046610375530309


In [211]:
def madhava_leibniz():
    a, sign = 1, 1
    while True:
        yield sign * (1 / a)
        a = a + 2
        sign = -sign

# for i in itertools.islice(madhava_leibniz(), 10):
#     print(i)

4 * sum(itertools.islice(madhava_leibniz(), 1000000))

3.1415916535897934

In [212]:
## Collections

In [213]:
import collections

In [214]:
s = (1, 2, 3)

In [216]:
s[2]

3

In [225]:
Record = collections.namedtuple("Record", ["a", "b", "c"])

In [232]:
r = Record(c=3, b=2, a=1)

In [233]:
r._asdict()

{'a': 1, 'b': 2, 'c': 3}

In [234]:
### dataclasses

In [235]:
from dataclasses import dataclass

In [238]:
@dataclass
class InventoryItem:
    name: str
    unit_price: float
    quantity_on_hand: int = 0

    def total_cost(self) -> float:
        return self.unit_price * self.quantity_on_hand


* custom dictionaries

In [239]:
counter = collections.Counter()

In [240]:
dd = dict()

try:
    dd["a"] += 1
except KeyError:
    dd["a"] = 1

In [241]:
dd

{'a': 1}

In [242]:
try:
    dd["a"] += 1
except KeyError:
    dd["a"] = 1

In [243]:
dd

{'a': 2}

In [244]:
counter = collections.Counter()

In [245]:
counter["a"] += 1

In [247]:
counter["a"] += 2

In [248]:
counter["b"] = 1

In [249]:
counter

Counter({'a': 3, 'b': 1})

In [270]:
counter.most_common(n=1)

[('a', 3)]

In [266]:
dd = collections.defaultdict(list)

In [267]:
dd["a"].append("hello")
dd["a"].append("world")

In [268]:
dd["a"]

['hello', 'world']

## Task

Write a short function that collects metrics over a text, e.g. how often each word occured. [str.split](https://docs.python.org/3.3/library/stdtypes.html#str.split) may be helpful.

```python
s = "hello hello world"

...

hello 2
world 1
```



In [273]:
import this

The Zen of Python, by Tim Peters

Beautiful is better than ugly.
Explicit is better than implicit.
Simple is better than complex.
Complex is better than complicated.
Flat is better than nested.
Sparse is better than dense.
Readability counts.
Special cases aren't special enough to break the rules.
Although practicality beats purity.
Errors should never pass silently.
Unless explicitly silenced.
In the face of ambiguity, refuse the temptation to guess.
There should be one-- and preferably only one --obvious way to do it.
Although that way may not be obvious at first unless you're Dutch.
Now is better than never.
Although never is often better than *right* now.
If the implementation is hard to explain, it's a bad idea.
If the implementation is easy to explain, it may be a good idea.
Namespaces are one honking great idea -- let's do more of those!


In [275]:
import collections
def count_words(text):
    
    separated_words = text.split(" ") # split()
    counter = collections.Counter()
    
    for word in separated_words:
        counter[word] += 1

    print(counter)

text = "text hallo welt das test text noch mal das hallo"

count_words(text)

Counter({'text': 2, 'hallo': 2, 'das': 2, 'welt': 1, 'test': 1, 'noch': 1, 'mal': 1})


In [276]:
import collections


def collect_words(text: str):
    list = str.split(text, " ")
    return collections.Counter(list)


print(collect_words("hello hello world"))

Counter({'hello': 2, 'world': 1})


In [277]:
def count_words(text: str):
    return collections.Counter(
        text.replace(".","").split()
    )

text = """
The Zen of Python, by Tim Peters
Beautiful is better than ugly.
Explicit is better than implicit.
Simple is better than complex.
Complex is better than complicated.
Flat is better than nested.
Sparse is better than dense.
"""
print(count_words(text))

Counter({'is': 6, 'better': 6, 'than': 6, 'The': 1, 'Zen': 1, 'of': 1, 'Python,': 1, 'by': 1, 'Tim': 1, 'Peters': 1, 'Beautiful': 1, 'ugly': 1, 'Explicit': 1, 'implicit': 1, 'Simple': 1, 'complex': 1, 'Complex': 1, 'complicated': 1, 'Flat': 1, 'nested': 1, 'Sparse': 1, 'dense': 1})


In [278]:
import requests

In [279]:
resp = requests.get("https://www.gutenberg.org/cache/epub/32522/pg32522.txt")

In [280]:
len(resp.text)

82058

In [281]:
resp.text[:30]

'\ufeffThe Project Gutenberg eBook o'

In [282]:
c = collections.Counter(resp.text.lower().split())

In [284]:
c.most_common(n=20)

[('the', 873),
 ('of', 315),
 ('to', 305),
 ('a', 299),
 ('and', 234),
 ('you', 180),
 ('in', 171),
 ('he', 165),
 ('was', 132),
 ('kramer', 117),
 ('it', 116),
 ('i', 108),
 ('that', 104),
 ('at', 103),
 ('with', 101),
 ('*', 95),
 ('is', 91),
 ('this', 86),
 ('or', 86),
 ('project', 85)]

In [285]:
## Type hints

In [286]:
def hello(greeting: "greeting word", name: "custom name") -> "result":
    return f"{greeting} from {name}"

In [287]:

print(hello("hello", "world"))
print(hello.__annotations__)

hello from world
{'greeting': 'greeting word', 'name': 'custom name', 'return': 'result'}


In [288]:
def add(a: int, b: int) -> int:
    return a + b

In [289]:
add.__annotations__

{'a': int, 'b': int, 'return': int}

In [291]:
class Employee:
    pass

from typing import Mapping, Set

def notify_by_email(employees: Set[Employee], overrides: Mapping[str, str]) -> None:
    pass

In [292]:
from typing import Sequence, TypeVar

T = TypeVar('T')      # Declare type variable

def first(l: Sequence[T]) -> T:   # Generic function
    return l[0]

Example Project: https://github.com/miku/wettr